In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

from sklearn.decomposition import NMF
from scipy.sparse.linalg import svds

#set the max columns to none
pd.set_option('display.max_columns', None)

## I will perform matrix factorization to the interaction matrix

In [34]:
# Read interaction matrix pickle file
type_ = 'train'
fillna_value = "0"
size = 'demo'
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{type_}_{size}_{fillna_value}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)
print(interaction_matrix.shape)

(1590, 1114)


In [35]:
interaction_matrix.head(2)

merge_article_ids,9775489,9775567,9776715,9776855,9776897,9778627,9778701,9779860,9754160,9768308,9771187,9771351,9772343,9772453,9772869,9773768,9774404,9774430,9775142,9775171,9761586,9770028,9778375,9466422,9762122,9771126,9771367,9771572,9771626,9772029,9772485,9773018,9773673,9773852,9774574,9774764,9776385,9777475,9777636,9777811,9778110,9778328,9779007,9780020,9772284,9779713,9735909,9770882,9770082,9775785,9775964,9776303,9772434,9772866,9773493,9773612,9775983,9776442,9772545,9759966,9769800,9771168,9773364,9773887,9774595,9776322,9776489,9777164,9777992,9778277,9778351,9778413,9779511,9779577,9779737,9767470,9768043,9770729,9770886,9770997,9771042,9771254,9771554,9772050,9772302,9772380,9772389,9772668,9772835,9773316,9773447,9773644,9773846,9773987,9774096,9774229,9774352,9774506,9774532,9774598,9774618,9774937,9775202,9776046,9776176,9776278,9777292,9778074,9778381,9779288,9779747,9759955,9765941,9769370,9769497,9769996,9770568,9771166,9771479,9771627,9771842,9771938,9771948,9772088,9772115,9772219,9772279,9772557,9772635,9772957,9773351,9773617,9773727,9775056,9775388,9775461,9775793,9775800,9775881,9775905,9775998,9776071,9776148,9776449,9776688,9776882,9777321,9777393,9777941,9778326,9778661,9778669,9778722,9778731,9778804,9778843,9778875,9778971,9779045,9779674,9779705,9729924,9772095,9772168,9775855,9775939,9776041,9776087,9776234,9776259,9776386,9776394,9746810,9769433,9769504,9769581,9771121,9771235,9771473,9771775,9771855,9772099,9772300,9772502,9772710,9772805,9772873,9773296,9774074,9774321,9774542,9774554,9774826,9774840,9775183,9775803,9775804,9775846,9776023,9776049,9776553,9776691,9777464,9778168,9778310,9778902,9773470,9775809,9776420,9777648,9778198,9770045,9770292,9771124,9771127,9771579,9771686,9771764,9771846,9772830,9773070,9773376,9767989,9768866,9771916,9771995,9774557,9775500,9775611,9775621,9776566,9778302,9780019,9758424,9772442,9777520,9767490,9772032,9775716,9775722,9769624,9771113,9772227,9772228,9772407,9772433,9774516,9779797,9759544,9769769,9771325,9771333,9771576,9771779,9771796,9771999,9772000,9772026,9772706,9775361,9775493,9775703,9776184,9778710,9778917,9778939,9778942,9779423,9744403,9760046,9774392,9774708,9775347,9775648,9772632,9775323,9766757,9770451,9772291,9775485,9775990,9776246,9776808,9776864,9778007,9778021,9778386,9778448,9728166,9766140,9772263,9778745,9769605,9771859,9772344,9773543,9773857,9774013,9774382,9774652,9775419,9776040,9776337,9776474,9777768,9777822,9779629,9772629,9777299,9775894,9776223,9778623,9776583,9778257,8315213,9531745,9535187,9747757,9755800,9761859,9769917,9770029,9770989,9771568,9771896,9772104,9772659,9772711,9773210,9773566,9773726,9774252,9774527,9775402,9775776,9775915,9775985,9776070,9776292,9777026,9777505,9779417,9779520,9779867,9779956,9775903,9776985,9769762,9779263,9768177,9772082,9772045,9772193,9773292,9773295,9773356,9773947,9774403,9777565,9767665,9770283,9771128,9771996,9775673,9775908,9776191,9776918,9777804,9779204,9779366,9779377,9482380,9755712,9761926,9769341,9769531,9770288,9771065,9771125,9771242,9772275,9772517,9772575,9772772,9773985,9774032,9774363,9775277,9775331,9775568,9775582,9775596,9775717,9776341,9776544,9777358,9777385,9777492,9778192,9778813,9779427,9779748,9779777,9780039,9770538,9771869,9773015,9769432,9771350,9459174,9737094,9742173,9763188,9770369,9772241,9772660,9772727,9772750,9772768,9772882,9772923,9772925,9772963,9773045,9773084,9775003,9753689,9773282,9774020,9774079,9772297,9772443,9772355,9772601,9775562,9776917,9779184,9779285,9772543,9775909,9778780,9771248,9771438,9774789,9770450,9771253,9771919,9773341,9773877,9778236,9775701,9775927,9776870,9694850,9768850,9769712,9775873,9777319,9769306,9772047,9773464,9779498,9391394,9508202,9566489,9738729,9766136,9767342,9768469,9769424,9770541,9772178,9772448,9773228,9773275,9773412,9773434,9773488,9773962,9774559,9774923,9775042,9776332,9776369,9776508,9776560,9776929,9777233,9777307,9777582,9778666,9778682,9778769,9778788,9779186,9486080,9774383,977940

In [36]:
# ---------(NOT USED)-----------------
# # Apply matrix factorization 
# nmf = NMF(n_components=5)
# user_matrix = nmf.fit_transform(interaction_matrix)
# article_matrix = nmf.components_.T


# Perform Singular Value Decomposition (SVD) on the user-article interaction matrix
k = 300  # Number of latent factors (embedding dimensions)

U, sigma, Vt = svds(interaction_matrix.values, k=k)  # Perform SVD, obtaining U, sigma, Vt

# Diagonalize the singular values sigma to create a diagonal matrix
sigma_diag = np.diag(sigma)

# Reconstruct the user and article embeddings
user_matrix = np.dot(U, np.sqrt(sigma_diag))  # User embeddings
article_matrix = np.dot(np.sqrt(sigma_diag), Vt).T  # Article embeddings

In [37]:
print(user_matrix.shape)
print(article_matrix.shape)

(1590, 300)
(1114, 300)


In [38]:
interaction_matrix

merge_article_ids  9775489  9775567  9776715  9776855  9776897  9778627  \
user_id                                                                   
11313                 13.0    108.0     12.0      4.0     11.0     51.0   
13538                  0.0      0.0      0.0      0.0      0.0      0.0   
15430                  0.0      0.0      0.0      0.0      0.0      0.0   
19181                  0.0      0.0      0.0      0.0      0.0      0.0   
19568                  0.0      0.0      0.0      0.0      0.0      0.0   
...                    ...      ...      ...      ...      ...      ...   
2581162                0.0      0.0      0.0      0.0      0.0      0.0   
2583035                0.0      0.0      0.0      0.0      0.0      0.0   
2583477                0.0      0.0      0.0      0.0      0.0      0.0   
2585449                0.0      0.0      0.0      0.0      0.0      0.0   
2589391                0.0      0.0      0.0      0.0      0.0      0.0   

merge_article_ids  9778701  9779860  9754160  9768308  9771187  9771351  \
user_id                                                                   
11313                  9.0     21.0      0.0      0.0      0.0      0.0   
13538                  0.0      0.0     11.0      6.0     18.0     15.0   
15430                  0.0      0.0      0.0      0.0      0.0      0.0   
19181                  0.0      0.0      0.0      0.0      0.0      0.0   
19568                  0.0      0.0      0.0      0.0      0.0      0.0   
...                    ...      ...      ...      ...      ...      ...   
2581162               49.0      0.0      0.0      0.0      0.0      0.0   
2583035                0.0      0.0      0.0      0.0      0.0      0.0   
2583477                0.0      0.0      0.0      0.0      0.0      0.0   
2585449                0.0      0.0      0.0      0.0      0.0      0.0   
2589391                0.0      0.0      0.0      0.0      0.0      0.0   

merge_article_ids  9772343  9772453  9772869  9773768  9774404  9774430  \
user_id                                                                   
11313                  0.0      0.0      0.0      0.0      0.0      0.0   
13538                 25.0     16.0     28.0     12.0     23.0     10.0   
15430                  0.0      0.0      0.0      0.0      0.0      0.0   
19181                  0.0      0.0      0.0      0.0      0.0      0.0   
19568                  0.0      0.0      0.0      0.0      0.0      0.0   
...                    ...      ...      ...      ...      ...      ...   
2581162                0.0      0.0      0.0      0.0      0.0      0.0   
2583035                0.0      0.0      0.0     34.0      0.0      0.0   
2583477                0.0      0.0      0.0      0.0      0.0      0.0   
2585449                0.0      0.0      0.0      0.0      0.0      0.0   
2589391                0.0      0.0      0.0      0.0      0.0      0.0   

merge_article_ids  9775142  9775171  9761586  9770028  9778375  9466422  \
user_id                                                                   
11313                  0.0      0.0      0.0      0.0      0.0      0.0   
13538                 26.0      8.0      0.0      0.0      0.0      0.0   
15430                  0.0      0.0     21.0    146.0     12.0      0.0   
19181                  0.0      0.0      0.0      0.0      0.0     11.0   
19568                  0.0      0.0      0.0      0.0      0.0      0.0   
...                    ...      ...      ...      ...      ...      ...   
2581162                0.0      0.0      0.0      0.0      0.0      0.0   
2583035                0.0      0.0      0.0      0.0      0.0      0.0   
2583477                0.0      0.0      0.0      0.0      0.0      0.0   
2585449                0.0      0.0      0.0      0.0      0.0      0.0   
2589391                0.0      0.0      0.0      0.0      0.0      0.0   

merge_article_ids  9762122  9771126  9771367  9771572  9771626  9772029  \
user_id              

In [39]:
column_lst = interaction_matrix.columns.tolist()

user_matrix_df = pd.DataFrame(user_matrix, index=interaction_matrix.index)
article_matrix_df = pd.DataFrame(article_matrix, index=column_lst)

In [40]:
user_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11313,-0.070924,-0.217349,0.236625,-0.100848,-0.019888,0.148840,0.232078,0.353403,-0.040379,0.044742,0.240427,0.003835,0.289386,0.305738,0.262294,-0.053922,0.335100,-0.047852,0.236782,-0.041947,-0.022142,0.051854,0.239779,-0.053060,-0.011951,0.109297,-0.336618,-0.129487,-0.033395,-0.153427,-0.098989,-0.228751,0.001889,-0.028251,-0.256629,0.001160,-0.047469,0.074129,0.007033,-0.09141,-0.072831,-0.027259,-0.008884,-0.106552,0.074718,0.163811,-0.078683,0.158221,0.099104,0.191957,-0.112047,0.036980,-0.131926,-0.200829,-0.192700,-0.003310,0.065959,0.131713,-0.107513,0.005648,0.027327,-0.111315,-0.292026,0.054866,0.005303,0.080552,-0.046471,-0.002581,0.147374,-0.227467,0.08628,0.045157,0.009144,0.279343,0.088970,-0.123474,-0.056266,0.040446,0.006162,-0.126008,0.120074,-0.067592,0.090913,-0.034153,0.037271,0.010928,0.017006,-0.116537,0.037554,-0.073894,0.059182,-0.034927,0.021592,-0.042753,-0.001512,0.092429,0.111823,0.045256,-0.003285,-0.042441,0.002345,0.008559,-0.052780,0.026575,-0.029153,-0.096592,-0.028359,-0.001373,0.015289,-0.043924,-0.055858,0.043322,-0.028644,0.028752,0.089714,-0.031657,-0.005504,-0.089517,-0.065246,-0.042839,0.018139,-0.100116,-0.118791,-0.077234,-0.012902,0.033208,-0.058638,0.054083,-0.052729,-0.012163,0.038105,0.132273,-0.069961,0.026114,-0.041253,-0.000256,-0.040258,-0.044033,-0.001835,-0.060304,0.163150,-0.005859,0.032900,0.073327,-0.051641,0.007122,-0.005027,0.032436,0.016389,0.011326,-0.077696,-0.041304,-0.118156,-0.021822,-0.023992,-0.005724,0.036346,0.085066,-0.046670,-0.081147,-0.012558,0.008829,0.025679,0.033235,0.013397,-0.011422,-0.021070,-0.014541,-0.137683,-0.080976,-0.019569,-0.035248,-0.045700,-0.040087,0.060749,0.003333,0.010693,-0.002936,-0.056511,0.043077,-0.021012,0.108037,-0.004470,0.019514,-0.015507,0.025936,0.025355,-0.014519,-0.000398,0.031459,0.016086,-0.142637,0.038185,0.027196,-0.268849,0.011378,0.085432,-0.045981,0.039207,0.088381,-0.004568,0.021127,-0.010565,-0.011808,0.019081,0.017888,-0.106918,0.054715,-0.008269,0.023820,0.013034,0.000368,0.019591,-0.037909,0.007301,-0.031141,-0.014904,0.006898,0.004260,0.000228,-0.013717,-0.003827,0.012502,-0.014020,-0.026040,0.030252,-0.051205,0.012709,0.017543,-0.010062,0.034242,0.016685,0.021230,-0.000774,0.025017,-0.036244,-0.012218,0.013188,-0.003938,0.001523,-0.029880,-0.005535,-0.001937,0.005162,0.014864,0.004207,0.004876,0.028403,-0.031504,0.020265,-0.017206,-0.050879,-0.085258,-0.012247,-0.004897,-0.014565,0.003239,0.043348,0.000903,0.001195,0.003866,-0.047988,0.029643,0.000467,0.000705,0.002931,-0.008606,0.007849,0.004696,-0.011516,-0.003338,-0.000126,0.004182,

In [41]:
article_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
9775489,0.366531,0.146713,-0.164704,0.04036,-0.173250,0.126776,-0.270314,0.193243,-0.373554,-0.752814,0.047475,0.165447,0.369539,-0.101192,0.314316,-0.752140,0.196385,-0.353437,0.060577,0.027135,-0.006802,-0.037281,-0.160731,-0.429633,0.328061,0.092270,-0.324022,-0.095514,0.565548,-0.353144,0.128313,0.128343,0.357816,-0.500564,0.398039,0.042621,-0.608196,-0.260849,-0.244545,-0.336135,-0.256256,-0.140131,0.072897,-0.279039,-0.186994,-0.225825,-0.025249,0.248133,0.530325,0.79876,-0.220345,-0.065665,-0.236364,0.260522,0.139165,-0.238614,-0.275968,0.136848,-0.238195,-0.091416,0.198777,-0.471227,-0.483159,0.165991,0.280870,-0.135572,-0.402420,-0.054765,-0.057944,0.141433,0.731667,-0.121944,-0.514119,0.302942,-0.461601,-0.403174,0.139571,-0.404738,-0.225726,0.147261,0.274024,-0.054290,0.035754,0.390836,-0.123068,0.519605,-0.031912,-0.503094,-0.22284,-0.52215,0.251610,-0.202184,0.207929,-0.042944,-0.036644,-0.289075,0.394927,-0.132263,0.034356,0.442255,-0.075544,0.098178,0.008806,0.246458,-0.471797,0.122896,0.221108,-0.178237,-0.073651,-0.416437,-0.256581,-0.198534,-0.001866,-0.186491,-0.222237,0.075504,-0.471199,-0.066568,-0.531980,0.087106,0.006361,-0.310222,-0.119030,0.127764,-0.092998,-0.212865,-0.051454,0.000081,-0.060009,0.136156,0.667128,0.659261,-0.453594,0.208585,-0.025321,0.226773,-0.265982,0.144314,-0.150977,0.318086,0.276051,-0.168555,0.143250,0.146344,-0.553092,0.095921,0.362733,0.612400,0.529367,-0.054540,-0.504430,0.182605,-0.806398,-0.355462,-0.485379,0.282488,0.175514,0.301834,0.155610,0.56720,0.016310,0.180202,-0.277237,-0.446110,-0.170939,-0.529385,-0.208979,0.06521,-0.127126,0.203816,-0.333172,0.409562,-0.06793,-0.090127,0.239588,0.561578,-0.483282,-0.034103,-0.583439,0.668684,0.590223,-0.269106,-0.211099,0.485310,-0.169552,0.791563,-1.041376,-1.125098,-0.017063,0.010853,-0.079023,-0.152378,-0.176040,0.165825,0.018576,0.599160,-0.529705,0.206101,-0.682822,0.860465,0.353694,0.312030,0.177544,0.001453,0.041945,-0.804464,-0.861326,-0.497856,0.887977,-0.060079,-0.365692,0.226573,-1.124282,-1.237913,0.272116,-0.931685,-1.940653,1.177726,-0.457293,-0.480756,-0.220827,0.313174,-0.542262,0.328551,0.299329,0.908449,-0.410503,0.105612,0.721629,-0.018268,0.186579,-0.120131,-0.009169,-0.979355,0.306849,-0.063508,-0.119972,-0.019993,-0.010634,-0.049488,-0.042178,0.06918,0.001981,-0.116866,-0.081509,0.038860,0.090636,-0.17216,-0.145934,0.262978,-0.588980,0.167754,-0.066593,-0.415109,0.062145,0.092193,-0.021404,0.163509,0.011843,-0.017060,0.158971,-0.050876,0.079993,-0.125434,0.133076,-0.106695,0.072380,0.074110,0.028051,0.008444,-0.044649,-0.126802,-0.091788,0.158369,-0.065857,0.387346,0.021664,-0.116307,0.062242,0.131878,-0.034940,0.121076,-0.012833,-0.041163,-0.125218,-0.158886,0.143187,-0.028119,0.079310,-0.149353,-0.047949,0.048509,0.022732,0.017720,0.017944,0.265457,0.341005,0.011712,-0.146772,0.08470
9775567,-0.158633,-0.051683,0.110599,-0.19692,0.0244

In [42]:
# Save the cosine similarity matrix to pickle file cause is very big

user_matrix_df_file_path = f'./files/pickle/user_matrix_{type_}_{size}_{fillna_value}.pkl'
article_matrix_df_file_path = f'./files/pickle/article_matrix_{type_}_{size}_{fillna_value}.pkl'

user_matrix_df.to_pickle(user_matrix_df_file_path)
article_matrix_df.to_pickle(article_matrix_df_file_path)

## Other

In [43]:
# # Define a function to convert strings back to lists
# def str_to_list(s):
#     try:
#         temp_lst = s.replace('[', '').replace(']','').replace(' ', '').split(',')
        
#         return [float(x) for x in temp_lst]
#     except:
#         return s


# for col in tqdm(interaction_matrix.columns.to_list()):
#     interaction_matrix[col] = interaction_matrix[col].apply(str_to_list)